In [ ]:
import pathlib
import os
import pandas as pd
import joblib
import yaml
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
print("Running train component")

In [ ]:
data = "data_train.csv"
output_model = "model.joblib"

param_file = "params.yml"

create_output_directories = True

# Could add TfidfTransformer settings here too

In [ ]:
with open(param_file, 'r') as fin:
    params = yaml.safe_load(fin)

svm_gamma = params['svm_gamma']
svm_c = params['svm_c']

In [ ]:
# Create directory for output files if not exists
# (required as kubeflow pipelines may pass a temp filename with 
# parents that do not exist yet)
if create_output_directories:
    for fname in [output_model]:
        print(f"Creating directory for output file {fname}")
        dir_path = os.path.dirname(os.path.realpath(fname))
        dir_path = pathlib.Path(dir_path)
        dir_path.mkdir(parents=True, exist_ok=True)
        print(f"Directory created: {dir_path}")

In [ ]:
FEATURE = 'text_feature'
TARGET = 'target'

In [ ]:
print("Reading training data")
df = pd.read_csv(data)

In [ ]:
pl = Pipeline(
    steps=[
        ("tfidf", TfidfVectorizer(stop_words='english')),
        ("svm", SVC(C=svm_c, gamma=svm_gamma, random_state=42))
    ]
)

In [ ]:
X = df[FEATURE].to_numpy()

In [ ]:
y = df[TARGET].to_numpy()

In [ ]:
print("training pipeline")
pl.fit(X, y)

In [ ]:
print(f"Saving output model to {output_model}")
joblib.dump(pl, output_model, compress=3)